# Image Classification for Rock Paper Scissors Moves

Author: Madison Morgan, 2020

Here is a notebook that demos two different types of deep learning image classifiers.
For Rock Paper Scissors Moves to be used in an app. Here is demonstrated and tested:
    A developed CNN model and a MobileVNET model.
    
Credits: I referenced a few notebooks, tutorials, and data 
* https://colab.research.google.com/github/trekhleb/machine-learning-experiments/blob/master/experiments/rock_paper_scissors_mobilenet_v2/rock_paper_scissors_mobilenet_v2.ipynb#scrollTo=DJ8jGFnTLt8t
* https://www.tensorflow.org/tutorials/images/data_augmentation

# Table of Contents

* 1.0 [Importing Packages and Defining Constants](#1.0)
* 2.0 [Data Preprocessing](#2.0)
    * 2.1 [Data Preprocessing: CNN Model](#2.1)
    * 2.2 [Data Preprocessing: MobileNetV2](#2.2)
* 3.0 [Transforming the Data](#3.0)
* 4.0 [Augmenting the Data](#4.0)
* 5.0 [Preparing the Data for Model](#5.0)
    * 5.1 [Defining Data Generators](#5.1)
    * 5.2 [Shuffling the Data](#5.2)
* 6.0 [Defining the Models](#6.0)
    * 6.1[Defining the Models: CNN Model](#6.1)
    * 6.2[Defining the Models: MobileNetV2](#6.2)
* 7.0 [Training the Models](#7.0)
    * 7.1[Training the Models: CNN Model](#7.1)
    * 7.2[Training the Models: MobileNetV2](#7.2)
* 8.0 [Evaluating Accuracy and Loss of Model](#8.0)
* 9.0 [Testing the Model](#9.0)
* 10.0 [Saving Model as TF.Lite for App](#10.0)

# Importing Packages and Defining Constants <a class="anchor" id="1.0"></a>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
import shutil
import tensorflow as tf
%matplotlib inline

In [ ]:
#Define some constants
DATA_PATH ="C://Users//haide//Desktop//MadyProjectz//RPS//Shoot//downloaded//"
TRAIN_PATH = "C://Users//haide//Desktop//MadyProjectz//RPS//Shoot//data//train//"
TEST_PATH = "C://Users//haide//Desktop//MadyProjectz//RPS//Shoot//data//test//"
VALIDATION_PATH = "C://Users//haide//Desktop//MadyProjectz//RPS//Shoot//data//validation//"
BEST_MODEL_PATH = "best_model.h5"
INPUT_SHAPE = (128,128,3)
TARGET_SIZE = (128,128)
BATCH_SIZE = 32
CLASS_MODE = 'categorical'
TF_DATASET = 'rock_paper_scissors'
TRAINING_SIZE = 0.9 #train CNN with 90% of images
TESTING_SIZE = 0.05 #test CNN with 5% of images
VALIDATION_SIZE = 0.05 #validate CNN with 5% of images

# Data Preprocessing <a class="anchor" id="2.0"></a>
Importing the data and seeing what the heck it looks like

## Data Preprocessing: CNN Model <a class="anchor" id="2.1"></a>
Need only run once, hence commented out to avoid accidentally rerunning

In [ ]:
#print(len(os.listdir(DATA_PATH+"/paper")))
#print(len(os.listdir(DATA_PATH+"/rock")))
#print(len(os.listdir(DATA_PATH+"/scissors")))

#def moveFiles(files,src,dest):
 #   for file in files:
  #      full_file_name = os.path.join(src, file)
   #     if os.path.isfile(full_file_name):
    #        shutil.copy(full_file_name,dest)


#def separateData():
 #   for move in ['paper','rock','scissors']:
  #      files = os.listdir(DATA_PATH+move)
   #     train_index = math.ceil(len(files)*0.9)
    #    test_index = train_index+ math.ceil(len(files)*0.05)
     #   validation_index = test_index+ math.floor(len(files)*0.05)
    
      #  moveFiles(files[:train_index],DATA_PATH+move,TRAIN_PATH+move)
       # moveFiles(files[train_index:test_index],DATA_PATH+move,TEST_PATH+move)
        #moveFiles(files[test_index:validation_index],DATA_PATH+move,VALIDATION_PATH+move)
    
    
    
    
    
    

## Data Preprocessing: MobileNetV2 <a class="anchor" id="2.2"></a>

In [ ]:
(dataset_train_raw, dataset_test_raw), dataset_info = tfds.load(
    name=TF_DATASET,
    data_dir='tmp',
    with_info=True,
    as_supervised=True,
    split=['train','test'])

In [ ]:
print('Raw train dataset size:', len(list(dataset_train_raw)),)
print('Raw test dataset size:', len(list(dataset_test_raw)), '\n')

In [ ]:
# For Mobilenet v2 possible input sizes are [96, 128, 160, 192, 224].
INPUT_IMG_SIZE_REDUCED = 224
INPUT_IMG_SHAPE_REDUCED = (
    INPUT_IMG_SIZE_REDUCED,
    INPUT_IMG_SIZE_REDUCED,
    INPUT_SHAPE[2]
)
INPUT_SIZE= INPUT_IMG_SIZE_REDUCED
print(str(dataset_info.features['label'].int2str(0)))
print(dataset_info.features['image'].shape)

# Transforming the Data <a class="anchor" id="3.0"></a>

In [ ]:
def format_example(image, label):
    # Make image color values to be float.
    image = tf.cast(image, tf.float32)
    # Make image color values to be in [0..1] range.
    image = image / 255.
    # Make sure that image has a right size
    image = tf.image.resize(image, [INPUT_SIZE, INPUT_SIZE])
    return image, label

#dataset_train = dataset_train_raw.map(format_example)
#dataset_test = dataset_test_raw.map(format_example)
print(list(dataset_train.take(1))[0])

# Augmenting the Data <a class="anchor" id="4.0"></a>

In this way we avoid overfitting the model and are able to generalize the model to a broader set of examples.

Consider when the image is horizontal, if the background is not bright, if the User uses the left hand? To adapt the model to more real life scenarios we can flip, rotate, and adjust the background colors of the images

In [ ]:
def augment(image,label):
    
    def augment_flip(image):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        return image
    
    def augment_color(image):
        image = tf.image.random_hue(image, max_delta=0.08)
        image = tf.image.random_saturation(image, lower=0.7, upper=1.3)
        image = tf.image.random_brightness(image, 0.05)
        image = tf.image.random_contrast(image, lower=0.8, upper=1)
        image = tf.clip_by_value(image, clip_value_min=0, clip_value_max=1)
        return image
    
    def augment_rotate(image):
        # Rotate 0, 90, 180, 270 degrees
        return tf.image.rot90(
        image,
        tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    
    def augment_invert(image):
        random = tf.random.uniform(shape=[], minval=0, maxval=1)
        if random > 0.5:
            image = tf.math.multiply(image, -1)
            image = tf.math.add(image, 1)
        return image
    
    def augment_zoom(image):
        image_width, image_height, image_colors = image.shape
        crop_size = (image_width, image_height)
        min_zoom, max_zoom = 0.8, 1.0
        
        # Generate crop settings, ranging from a 1% to 20% crop.
        scales = list(np.arange(min_zoom, max_zoom, 0.01))
        boxes = np.zeros((len(scales), 4))

        for i, scale in enumerate(scales):
            x1 = y1 = 0.5 - (0.5 * scale)
            x2 = y2 = 0.5 + (0.5 * scale)
            boxes[i] = [x1, y1, x2, y2]

        def random_crop(img):
            # Create different crops for an image
            crops = tf.image.crop_and_resize(
            [img],
            boxes=boxes,
            box_indices=np.zeros(len(scales)),
            crop_size=crop_size)
            # Return a random crop
            return crops[tf.random.uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]

        choice = tf.random.uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)

        # Only apply cropping 50% of the time
        return tf.cond(choice < 0.5, lambda: image, lambda: random_crop(image))
     
        
    image = augment_flip(image)
    image = augment_color(image)
    image = augment_rotate(image)
    image = augment_zoom(image)
    image = augment_invert(image)
    return image, label



train_data_augmented = dataset_train.map(augment)


# Preparing the Data for Model <a class="anchor" id="5.0"></a>

## Defining Data Generators <a class="anchor" id="5.1"></a>

In [ ]:




data_generator = ImageDataGenerator(rescale=1. / 255,
                            rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

validation_generator = ImageDataGenerator(rescale=1./255)

train_generator = data_generator.flow_from_directory(
    TRAIN_PATH,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=CLASS_MODE
)


test_generator = data_generator.flow_from_directory(
    TEST_PATH,
    target_size=TARGET_SIZE,
    batch_size=1,
    class_mode=CLASS_MODE
)




validation_generator = validation_generator.flow_from_directory(
    VALIDATION_PATH,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=CLASS_MODE
)




## Shuffling the Data <a class="anchor" id="5.2"></a>

In [ ]:
BATCH_SIZE = 800

dataset_train_augmented_shuffled = dataset_train_augmented.shuffle(
    buffer_size=NUM_TRAIN_EXAMPLES
)

dataset_train_augmented_shuffled = dataset_train_augmented.batch(
    batch_size=BATCH_SIZE
)

# Prefetch will enable the input pipeline to asynchronously fetch batches while your model is training.
dataset_train_augmented_shuffled = dataset_train_augmented_shuffled.prefetch(
    buffer_size=tf.data.experimental.AUTOTUNE
)

dataset_test_shuffled = dataset_test.batch(BATCH_SIZE)
batches = tfds.as_numpy(dataset_train_augmented_shuffled)

# Defining the Model <a class="anchor" id="6.0"></a>

## Defining the Model: CNN Model <a class="anchor" id="6.1"></a>

In [ ]:
class CNN:
    def __init__(self, dataPath, inputShape):
        self.datapath = dataPath
        self.model = self.build_model(inputShape)
        self.model.compile(loss='categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])
        
    
    def build_model(self, inputShape):
        model = Sequential()
        model.add(Conv2D(64, (3, 3), input_shape=inputShape, activation='relu'))
        #model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        #model.add(Dropout(0.2))

        model.add(Conv2D(128, (3, 3), activation='relu'))
        #model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        #model.add(Dropout(0.2))

        model.add(Flatten())
        model.add(Dropout(0.5))
        model.add(Dense(512, activation='relu'))
        model.add(Dense(3, activation='softmax'))
        
        model.summary()
        return model

In [ ]:
CNN = CNN(DATA_PATH, INPUT_SHAPE)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    BEST_MODEL_PATH, 
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

reduce_callback  = ReduceLROnPlateau(
    monitor = 'val_accuracy',
    patience = 3,
    factor = 0.5,
    min_lr = 0.00001,
    verbose = 1
)

callbacks_list = [checkpoint_callback, reduce_callback]

## Defining the Model: MobileNetV2 for Feature Extraction <a class="anchor" id="6.2"></a>

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
  input_shape=INPUT_SHAPE,
  include_top=False,
  weights='imagenet',
  pooling='avg'
)

# Freezing base model, dont want to retrain only want feature extraction!
base_model.trainable= False
base_model.summary()

In [ ]:
tf.keras.utils.plot_model(
base_model,
show_shapes=True,
show_layer_names=True)

### Adding a Classification Head

In [ ]:
model = tf.keras.models.Sequential()

model.add(base_model)

# model.add(tf.keras.layers.GlobalAveragePooling2D())

model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(
    units=NUM_CLASSES,
    activation=tf.keras.activations.softmax,
    kernel_regularizer=tf.keras.regularizers.l2(l=0.01)
))

model.summary()
tf.keras.utils.plot_model(
    model,
    show_shapes=True,
    show_layer_names=True,
)

### Compiling the Model

In [ ]:
rmsprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

model.compile(
    optimizer=rmsprop_optimizer,
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

# Training the Model <a class="anchor" id="7.0"></a>

## Training the Model : CNN Model <a class="anchor" id="7.1"></a>

In [ ]:
step_size_train = train_generator.n//train_generator.batch_size
step_size_val = validation_generator.n//validation_generator.batch_size
CNN.history = CNN.model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=25,
                   validation_data=validation_generator,
                   validation_steps=step_size_val,
                   verbose=1,
                   callbacks=callbacks_list)

## Training the Model : MobileNetV2 <a class="anchor" id="7.2"></a>

# Evaluating Accuracy and Loss of Model <a class="anchor" id="8.0"></a>

In [ ]:
accuracy = CNN.history.history['acc']
validation_accuracy = CNN.history.history['val_acc']
loss = CNN.history.history['loss']
validation_loss = CNN.history.history['val_loss']

num_epochs = range(len(accuracy))
plt.plot(num_epochs, accuracy, 'r', label='Training Accuracy')
plt.plot(num_epochs, validation_accuracy, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
plt.plot(num_epochs, loss, 'r', label='Training Loss')
plt.plot(num_epochs, validation_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend(loc=0)
plt.figure()
plt.show()

# Testing the Model <a class="anchor" id="9.0"></a>

In [ ]:
CNN.model.load_weights(BEST_MODEL_PATH)

In [ ]:
step_size_test = test_generator.n//test_generator.batch_size
testing_model = CNN.model.evaluate_generator(
    test_generator,
    step_size_test,
    verbose=1
)

In [ ]:
classes = CNN.model.predict_generator(test_generator, test_generator.n)

In [ ]:
print(classes)

In [ ]:
len(classes)

In [ ]:
test_generator.classes

In [ ]:
len(test_generator.classes)

In [ ]:
from keras.preprocessing import image
from PIL import Image
import cv2
path = "C://Users//haide//Desktop//MadyProjectz//RPS//Shoot//data///testNEWPHOTOS//"
for file in os.listdir(path):
    img = Image.open(path+file)#, target_size(60,60))
    #print(img.mode)
    img.load()
    background = Image.new("RGB", img.size, (255,255,255))
    background.paste(img,mask=img.split()[3])
    img = background
    img = img.resize((60,60)) 
    img = image.img_to_array(img)
    #print(img.shape)
    imgplot = plt.imshow(img)
    img = img.reshape((1,)+img.shape)
    #print(img.shape)
    
    
    img_class = CNN.model.predict_classes(img)
    print("File: "+str(file)+", pred: "+str(img_class))

In [ ]:
print(img_class)

# Saving Model as TF.Lite for App <a class="anchor" id="10.0"></a>

In [ ]:
model = tf.keras.models.load_model("best_model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)